In [1]:
import torch
import numpy as np
from nemo.util.plotting import plot_surface
from nemo.field import NeuralHeightField

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

In [2]:
dem_path = "../../data/Moon_Map_01_0_rep0.dat"
dem = np.load(dem_path, allow_pickle=True)

In [ ]:
plot_surface(dem)

In [45]:
# Reshape DEM data from (180, 180, 3) to (N, 3) where N = 180*180
dem_reshaped = dem[:, :, :3].reshape(-1, 3)  # Flatten spatial dimensions

# Convert to torch tensor and move to CUDA
dem_tensor = torch.from_numpy(dem_reshaped).float().to(device)

# Split into input (x,y) and target (z) tensors
xy = dem_tensor[:, :2]  # First two columns are x,y coordinates
z = dem_tensor[:, 2]  # Third column is the height value
xyz = torch.cat([xy, z.unsqueeze(1)], dim=1)

In [46]:
field = NeuralHeightField(in_dim=2, encoding_type="nerf").to(device)

In [ ]:
field.fit(xyz)

In [49]:
field_z = field(xy)
field_dem = dem.copy()
field_dem[:, :, 2] = field_z.reshape(180, 180).detach().cpu().numpy()

In [ ]:
plot_surface(field_dem)